In [4]:
max_len = 1004

In [1]:
import pandas as pd
import numpy as np

xTrain = pd.read_csv("xTrain.csv", index_col=0)
xTest = pd.read_csv("xTest.csv", index_col=0)
yTrain = pd.read_csv("yTrain.csv", index_col=0)
yTest = pd.read_csv("yTest.csv", index_col=0)

In [7]:
set.union(*xTrain.Seq.map(set).to_list())

{'A', 'C', 'G', 'N', 'T'}

In [8]:
set.union(*xTest.Seq.map(set).to_list())

{'A', 'C', 'G', 'T'}

In [14]:
xTrain[xTrain.Seq.str.contains("N")].Seq.to_list()

['TTTCTGAGATCTGCTTTTAGTGAAGTGGATCAATGATGAAACTAGCCAAATCTGAGCATCAGAAGTCTTTCCAGTCTACCTGATGCATGATCTCTACAGTTCTGAGAAGCAAAACTATAAAACAATGTAAAACAATAAGGGCATATGTCTGGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTACGCACATGTGTTTATAAAGATAACAGCTGTAGGAATGAATGAGATTGAGGGTGGGGGGGTGCGTATGTATGTCTATGAAAGCCTAATCNTTTCTGGGCAATGATGAAAAGGTTTTACTACTGATCTTTGTAACTATGATGGTTTCTACACTTGACCTGAGCTCA']

In [5]:
from keras.preprocessing.sequence import pad_sequences

xTrain_pad = pad_sequences(xTrain.Seq.map(lambda x: [ord(c) for c in x]), maxlen=max_len)
xTest_pad = pad_sequences(xTest.Seq.map(lambda x: [ord(c) for c in x]), maxlen=max_len)

In [6]:
np.concatenate((xTrain_pad, xTest_pad))

array([[ 0,  0,  0, ..., 84, 84, 84],
       [ 0,  0,  0, ..., 84, 67, 67],
       [ 0,  0,  0, ..., 84, 67, 65],
       ...,
       [ 0,  0,  0, ..., 84, 67, 65],
       [ 0,  0,  0, ..., 65, 65, 65],
       [ 0,  0,  0, ..., 71, 84, 65]], dtype=int32)

In [39]:
from sklearn.preprocessing import OneHotEncoder

ohenc = OneHotEncoder(sparse=False, categories="auto")
ohenc.fit(np.concatenate((xTrain_pad, xTest_pad)))

OneHotEncoder(categorical_features=None, categories='auto', drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=False)

In [43]:
ohenc.categories_

[array([ 0, 84], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 67], dtype=int32),
 array([ 0, 67], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 71], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 71], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 67], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 71], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 71], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 67], dtype=int32),
 array([ 0, 67], dtype=int32),
 array([ 0, 67], dtype=int32),
 array([ 0, 67], dtype=int32),
 array([

In [40]:
print(ohenc.transform(xTrain_pad))

[[1. 0. 1. ... 0. 0. 1.]
 [1. 0. 1. ... 1. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 1. ... 0. 1. 0.]
 [1. 0. 1. ... 0. 0. 1.]
 [1. 0. 1. ... 1. 0. 0.]]


In [41]:
ohenc.transform(xTrain_pad).shape

(787, 3365)

In [44]:
ohenc.transform(xTest_pad).shape

(197, 3365)

In [42]:
xTrain_pad.shape

(787, 1004)

In [56]:
xTrain_oh = ohenc.transform(xTrain_pad)
xTrain_oh.shape

(787, 3365)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold


In [52]:
from sklearn.preprocessing import LabelEncoder

# convert labels to numbers 
encoder = LabelEncoder()
encoder.fit(yTrain.Class.to_list())
yTrain_enc = encoder.transform(yTrain.Class.to_list())

print(encoder.classes_)
print(encoder.transform(encoder.classes_))

['CD-box' 'HACA-box']
[0 1]


In [119]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils

def create_model_embedding(emb_in=6, emb_out=3, conv1_ks=16, conv1_w=3, conv2_ks=32, conv2_w=9, dense=13):
    # create the model
    model = keras.Sequential()
    model.add(keras.layers.Embedding(emb_in, emb_out, input_length=3365 , input_shape=(3365, ) ) )
    model.add(keras.layers.Conv1D(conv1_ks, conv1_w, activation='relu', padding='same'))
    #model.add(keras.layers.Conv1D(8, 3, activation='relu', padding='same', input_shape=(3365, 1)))
    model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.Conv1D(conv2_ks, conv2_w, activation='relu'))
    model.add(keras.layers.MaxPooling1D(2))
    #model.add(keras.layers.Conv1D(32, 16, activation='relu'))
    #model.add(keras.layers.Conv1D(32, 32, activation='relu'))
    #model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(dense, activation='relu'))
    #model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(2, activation=tf.nn.softmax))

    optAdam = keras.optimizers.Adam()
    model.compile(optimizer=optAdam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [115]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model_embedding, verbose=1)
#model.summary() #print model summary
model.fit(xTrain_oh, yTrain_enc, epochs=10, batch_size=32, verbose=1) # start training

Epoch 1/10
787/787 [==============================] - 1s 2ms/sample - loss: 0.6805 - acc: 0.7230
Epoch 2/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.5383 - acc: 0.8653
Epoch 3/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.3081 - acc: 0.8920
Epoch 4/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.2552 - acc: 0.9111
Epoch 5/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.2553 - acc: 0.9072
Epoch 6/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.2297 - acc: 0.9187
Epoch 7/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.2162 - acc: 0.9301
Epoch 8/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.2057 - acc: 0.9187
Epoch 9/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.1945 - acc: 0.9365
Epoch 10/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.1759 - acc: 0.9441


In [123]:
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_model_embedding, verbose=1)
# define the grid search parameters
batch_size = [32]
epochs = [5]
emb_in=[3,6,9] #6
emb_out=[2,3,4] #3
conv1_ks=[8,16,32] #16
conv1_w=[3,5]
conv2_ks=[8,16,32]
conv2_w=[7,9]
dense=[5,10,20,40]

param_grid = dict(batch_size=batch_size,
                 epochs=epochs,
                 emb_in=emb_in,
                 emb_out=emb_out,
                 conv1_ks=conv1_ks,
                 conv1_w=conv1_w,
                 conv2_ks=conv2_ks,
                 conv2_w=conv2_w,
                 dense=dense)



grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=2, n_jobs=-1, verbose=1)

grid_result = grid.fit(xTrain_oh, yTrain_enc)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 2 folds for each of 1296 candidates, totalling 2592 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/home/wehrmann/anaconda3/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 25.8min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 51.8min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 82.2min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 129.1min
[Parallel(n_jobs=-1)]: Done 2592 out of 2592 | elapsed: 141.0min finished
/home/wehrmann/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The defau

Epoch 1/5
787/787 [==============================] - 2s 3ms/sample - loss: 0.6987 - acc: 0.4828
Epoch 2/5
787/787 [==============================] - 2s 2ms/sample - loss: 0.6830 - acc: 0.7001
Epoch 3/5
787/787 [==============================] - 2s 2ms/sample - loss: 0.5211 - acc: 0.8463
Epoch 4/5
787/787 [==============================] - 2s 2ms/sample - loss: 0.2746 - acc: 0.9098
Epoch 5/5
787/787 [==============================] - 2s 2ms/sample - loss: 0.2315 - acc: 0.9250
Best: 0.921220 using {'batch_size': 32, 'conv1_ks': 16, 'conv1_w': 5, 'conv2_ks': 32, 'conv2_w': 9, 'dense': 40, 'emb_in': 6, 'emb_out': 3, 'epochs': 5}
0.739517 (0.136290) with: {'batch_size': 32, 'conv1_ks': 8, 'conv1_w': 3, 'conv2_ks': 8, 'conv2_w': 7, 'dense': 5, 'emb_in': 3, 'emb_out': 2, 'epochs': 5}
0.677255 (0.211338) with: {'batch_size': 32, 'conv1_ks': 8, 'conv1_w': 3, 'conv2_ks': 8, 'conv2_w': 7, 'dense': 5, 'emb_in': 3, 'emb_out': 3, 'epochs': 5}
0.673443 (0.207531) with: {'batch_size': 32, 'conv1_ks': 

In [124]:
grid_result.best_params_

{'batch_size': 32,
 'conv1_ks': 16,
 'conv1_w': 5,
 'conv2_ks': 32,
 'conv2_w': 9,
 'dense': 40,
 'emb_in': 6,
 'emb_out': 3,
 'epochs': 5}

In [125]:
model = grid_result.best_estimator_

In [126]:
yPred = model.predict(ohenc.transform(xTest_pad))
yPred_class = np.array([np.argmax(row) for row in yPred])

from sklearn.metrics import classification_report

print(classification_report( encoder.transform(yTest.Class.to_list()), yPred_class))

197/197 [==============================] - 0s 2ms/sample
              precision    recall  f1-score   support

           0       0.50      1.00      0.67        99
           1       0.00      0.00      0.00        98

    accuracy                           0.50       197
   macro avg       0.25      0.50      0.33       197
weighted avg       0.25      0.50      0.34       197



/home/wehrmann/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [81]:
len(xTrain_oh[0])

3365

In [91]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils

# create the model
model = keras.Sequential()
#model.add(keras.layers.Embedding(5, 3, input_length=3365 , input_shape=(3365, ) ) )
#model.add(keras.layers.Conv1D(8, 3, activation='relu', padding='same'))
model.add(keras.layers.Conv1D(8, 3, activation='relu', padding='same', input_shape=(3365, 1)))
model.add(keras.layers.MaxPooling1D(2))
model.add(keras.layers.Conv1D(16, 7, activation='relu'))
model.add(keras.layers.MaxPooling1D(2))
#model.add(keras.layers.Conv1D(32, 16, activation='relu'))
#model.add(keras.layers.Conv1D(32, 32, activation='relu'))
#model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(16, activation='relu'))
#model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2, activation=tf.nn.softmax))

optAdam = keras.optimizers.Adam()
model.compile(optimizer=optAdam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#print model summary
model.summary()

# Fit the model
model.fit(xTrain_oh, yTrain_enc, epochs=10, batch_size=32, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_58 (Conv1D)           (None, 3365, 8)           32        
_________________________________________________________________
max_pooling1d_40 (MaxPooling (None, 1682, 8)           0         
_________________________________________________________________
conv1d_59 (Conv1D)           (None, 1676, 16)          912       
_________________________________________________________________
max_pooling1d_41 (MaxPooling (None, 838, 16)           0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 13408)             0         
_________________________________________________________________
dense_36 (Dense)             (None, 32)                429088    
_________________________________________________________________
dense_37 (Dense)             (None, 2)                 66        
Total para

ValueError: Error when checking input: expected conv1d_58_input to have 3 dimensions, but got array with shape (787, 3365)

In [97]:
yPred = model.predict(ohenc.transform(xTest_pad)) 
yPred

array([[0.03793205, 0.96206796],
       [0.6027639 , 0.39723617],
       [0.04488635, 0.9551136 ],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.05762911, 0.9423709 ],
       [0.03640307, 0.963597  ],
       [0.6027639 , 0.39723617],
       [0.02395522, 0.9760448 ],
       [0.3024247 , 0.6975753 ],
       [0.36231595, 0.63768405],
       [0.09731042, 0.90268964],
       [0.05010566, 0.9498943 ],
       [0.02146999, 0.97853   ],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.49592173, 0.50407827],
       [0.06055814, 0.93944186],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.01738231, 0.9826177 ],
       [0.6027639 , 0.39723617],
       [0.05334512, 0.94665486],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.

In [94]:
yPred_class = np.array([np.argmax(row) for row in yPred])
yPred_class

array([1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1])

In [95]:
from sklearn.metrics import classification_report

print(classification_report( encoder.transform(yTest.Class.to_list()), yPred_class))

              precision    recall  f1-score   support

           0       0.88      0.92      0.90        99
           1       0.91      0.88      0.90        98

    accuracy                           0.90       197
   macro avg       0.90      0.90      0.90       197
weighted avg       0.90      0.90      0.90       197



# Playground best convs and stuff

## create numerically encoded version of train/test data

In [60]:
from sklearn.preprocessing import LabelEncoder

base_encoder = LabelEncoder()
base_encoder.fit(np.concatenate((xTrain_pad, xTest_pad), axis=None))

xTrain_num = np.array([base_encoder.transform(seq) for seq in xTrain_pad])
xTest_num = np.array([base_encoder.transform(seq) for seq in xTest_pad])

print(xTrain_num.shape)
xTrain_num.reshape(787,1004,1)

(787, 1004)


array([[[0],
        [0],
        [0],
        ...,
        [5],
        [5],
        [5]],

       [[0],
        [0],
        [0],
        ...,
        [5],
        [2],
        [2]],

       [[0],
        [0],
        [0],
        ...,
        [5],
        [2],
        [1]],

       ...,

       [[0],
        [0],
        [0],
        ...,
        [5],
        [3],
        [3]],

       [[0],
        [0],
        [0],
        ...,
        [3],
        [1],
        [5]],

       [[0],
        [0],
        [0],
        ...,
        [2],
        [1],
        [2]]])

In [61]:
xTrain_num

array([[0, 0, 0, ..., 5, 5, 5],
       [0, 0, 0, ..., 5, 2, 2],
       [0, 0, 0, ..., 5, 2, 1],
       ...,
       [0, 0, 0, ..., 5, 3, 3],
       [0, 0, 0, ..., 3, 1, 5],
       [0, 0, 0, ..., 2, 1, 2]])

## create numerically encoded version of train/test labels

In [17]:
from sklearn.preprocessing import LabelEncoder

# convert labels to numbers 
encoder = LabelEncoder()
encoder.fit(yTrain.Class.to_list())

yTrain_enc = encoder.transform(yTrain.Class.to_list())

print(encoder.classes_)
print(encoder.transform(encoder.classes_))

['CD-box' 'HACA-box']
[0 1]


## create our first model

In [75]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils

def create_model_num(dropout=0.5):
    # create the model
    model = keras.Sequential()
    #model.add(keras.layers.Embedding(6, 3, input_length=1004 ) )
    model.add(keras.layers.Conv1D(16, 6, activation='relu', padding='same', input_shape=(1004,1)))
    #model.add(keras.layers.Conv1D(16, 6, activation='relu', padding='same'))
    #model.add(keras.layers.Conv1D(8, 3, activation='relu', padding='same', input_shape=(3365, 1)))
    model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.Conv1D(32, 16, activation='relu'))
    #model.add(keras.layers.Conv1D(64, 32, activation='relu'))
    model.add(keras.layers.MaxPooling1D(2))
    #model.add(keras.layers.Conv1D(128, 16, activation='relu'))
    #model.add(keras.layers.Conv1D(32, 32, activation='relu'))
    #model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(20, activation='relu'))
    #model.add(keras.layers.Dense(20, activation='relu'))
    #model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    optAdam = keras.optimizers.Adam()
    model.compile(optimizer=optAdam, loss='binary_crossentropy', metrics=['accuracy'])
    
    print(model.summary())
    
    return model


## train our first model with numerically encoded sequences

In [76]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model_num, verbose=1)

#from keras.utils import plot_model
#plot_model(model, to_file='model.png')

model.fit(xTrain_num.reshape(787,1004,1), yTrain_enc, epochs=5, batch_size=32, verbose=1) # start training

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_40 (Conv1D)           (None, 1004, 16)          112       
_________________________________________________________________
max_pooling1d_31 (MaxPooling (None, 502, 16)           0         
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 487, 32)           8224      
_________________________________________________________________
max_pooling1d_32 (MaxPooling (None, 243, 32)           0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 7776)              0         
_________________________________________________________________
dense_42 (Dense)             (None, 20)                155540    
_________________________________________________________________
dense_43 (Dense)             (None, 1)                 21        
Total para

## test our first model with numerically encoded sequences

In [77]:
yPred = model.predict(xTest_num.reshape(-1,1004,1))
#yPred_class = np.array([np.argmax(row) for row in yPred])

from sklearn.metrics import classification_report

print(classification_report( yTest.Class.to_list(), encoder.inverse_transform(yPred) ))

197/197 [==============================] - 0s 1ms/sample
              precision    recall  f1-score   support

      CD-box       0.87      0.90      0.89        99
    HACA-box       0.89      0.87      0.88        98

    accuracy                           0.88       197
   macro avg       0.88      0.88      0.88       197
weighted avg       0.88      0.88      0.88       197



# One hot encoded sequences

In [124]:
from sklearn.preprocessing import OneHotEncoder

print(xTrain_num.shape)
print(xTrain_num[0].reshape(-1, 1))
xTrain_num_resh =  xTrain_num.T.reshape(1004,787)
#print(xTrain_num_resh.T)
onehotencoder = OneHotEncoder(categories='auto', dtype=int, sparse=False)
print( np.concatenate( (xTrain_num.reshape(-1, 1), xTest_num.reshape(-1, 1)), axis=0 ) )
onehotencoder.fit( np.concatenate( (xTrain_num.reshape(-1, 1), xTest_num.reshape(-1, 1)), axis=0 ) )

xTrain_ohe = np.array([onehotencoder.transform(seq.reshape(-1, 1)) for seq in xTrain_num])
xTest_ohe = np.array([onehotencoder.transform(seq.reshape(-1, 1)) for seq in xTest_num])

print(np.array(xTrain_ohe).shape)
#xTrain_ohe = np.array(xTrain_ohe)

(787, 1004)
[[0]
 [0]
 [0]
 ...
 [5]
 [5]
 [5]]
[[0]
 [0]
 [0]
 ...
 [3]
 [5]
 [1]]
(787, 1004, 6)


## create our second model (for one hot encoding)

In [122]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils

def create_model_ohe(dropout=0.5):
    # create the model
    model = keras.Sequential()
    #model.add(keras.layers.Embedding(6, 3, input_length=1004 ) )
    model.add(keras.layers.Conv1D(16, 6, activation='relu', padding='same', input_shape=(1004,6)))
    #model.add(keras.layers.Conv1D(16, 6, activation='relu', padding='same'))
    #model.add(keras.layers.Conv1D(8, 3, activation='relu', padding='same', input_shape=(3365, 1)))
    model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.Conv1D(32, 16, activation='relu'))
    #model.add(keras.layers.Conv1D(64, 32, activation='relu'))
    model.add(keras.layers.MaxPooling1D(2))
    #model.add(keras.layers.Conv1D(128, 16, activation='relu'))
    #model.add(keras.layers.Conv1D(32, 32, activation='relu'))
    #model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(20, activation='relu'))
    #model.add(keras.layers.Dense(20, activation='relu'))
    #model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    optAdam = keras.optimizers.Adam()
    model.compile(optimizer=optAdam, loss='binary_crossentropy', metrics=['accuracy'])
    
    print(model.summary())
    
    return model


## train second model with numerically encoded sequences

In [127]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model_ohe, verbose=1)

#from keras.utils import plot_model
#plot_model(model, to_file='model.png')

model.fit(xTrain_ohe, yTrain_enc, epochs=5, batch_size=32, verbose=1) # start training

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_46 (Conv1D)           (None, 1004, 16)          592       
_________________________________________________________________
max_pooling1d_37 (MaxPooling (None, 502, 16)           0         
_________________________________________________________________
conv1d_47 (Conv1D)           (None, 487, 32)           8224      
_________________________________________________________________
max_pooling1d_38 (MaxPooling (None, 243, 32)           0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 7776)              0         
_________________________________________________________________
dense_48 (Dense)             (None, 20)                155540    
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 21        
Total para

## test our first model with numerically encoded sequences

In [128]:
yPred = model.predict(xTest_ohe)
#yPred_class = np.array([np.argmax(row) for row in yPred])

from sklearn.metrics import classification_report

print(classification_report( encoder.transform(yTest.Class.to_list()), yPred))

197/197 [==============================] - 0s 1ms/sample
              precision    recall  f1-score   support

           0       0.86      0.95      0.90        99
           1       0.94      0.85      0.89        98

    accuracy                           0.90       197
   macro avg       0.90      0.90      0.90       197
weighted avg       0.90      0.90      0.90       197



In [12]:
import graphviz

In [217]:
n_values=5

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from tensorflow.keras.utils import to_categorical



ohenc = OneHotEncoder(sparse=False, categories="auto",dtype=int)
ohenc.fit(np.concatenate((xTrain_pad, xTest_pad)))

enc = ohenc.transform(xTrain_pad)
#print(list(xTrain_pad[0]))

samp = np.array([[0,71,71,71,84],[0,65,71,71,84]])
print(samp.shape, samp[0], list(set(samp[0])))
print(samp.T.shape, list(samp.T))

base_encoder = LabelEncoder()
base_encoder.fit(samp.flatten())
samp_num = np.array([base_encoder.transform(s) for s in samp])
samp_num

#ord_encoder =  OrdinalEncoder(dtype=int)
#samp_ord = ord_encoder.fit_transform(samp.T)
#print(samp_ord)
#samp2 = np.array([[0,1,0,0,0],[0,0,0,0,0]])
#ord_encoder.inverse_transform(samp2)

#samp.reshape(-1, 1)

onehotencoder = OneHotEncoder(categories='auto', dtype=int, sparse=False)

samp_num1 = samp_num[0].reshape(len(samp_num[0]), 1)
samp_oh = onehotencoder.fit_transform(samp_num1)
samp_oh
#enc = ohenc.transform(samp)

#print(list(enc[0]))
#enc_samp
#to_categorical(samp[0], num_classes=5)

(2, 5) [ 0 71 71 71 84] [0, 84, 71]
(5, 2) [array([0, 0]), array([71, 65]), array([71, 71]), array([71, 71]), array([84, 84])]


array([[1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [230]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from tensorflow.keras.utils import to_categorical
print(xTrain_pad.flatten())
print(np.concatenate((xTrain_pad, xTest_pad), axis=None))

[ 0  0  0 ... 67 65 67]
[ 0  0  0 ... 71 84 65]


In [334]:
base_encoder = LabelEncoder()
base_encoder.fit(np.concatenate((xTrain_pad, xTest_pad), axis=None))

xTrain_num = np.array([base_encoder.transform(seq) for seq in xTrain_pad])
xTest_num = np.array([base_encoder.transform(seq) for seq in xTest_pad])
xTrain_num

array([[0, 0, 0, ..., 5, 5, 5],
       [0, 0, 0, ..., 5, 2, 2],
       [0, 0, 0, ..., 5, 2, 1],
       ...,
       [0, 0, 0, ..., 5, 3, 3],
       [0, 0, 0, ..., 3, 1, 5],
       [0, 0, 0, ..., 2, 1, 2]])

In [328]:
#xTrain_prep = np.array([seq.reshape(len(seq), 1) for seq in xTrain_num])
#print(xTrain_prep.shape)
print(xTrain_num.shape)
print(list(xTrain_num[0]))
print(xTrain_num[0].reshape(1, -1) )
onehotencoder = OneHotEncoder(categories='auto', dtype=int, sparse=False)
onehotencoder.fit(xTrain_num[0].reshape(1, -1))

#xTrain_ohe = [onehotencoder.transform(seq) for seq in xTrain_num]

#print(list(xTrain_ohe[0]))
onehotencoder.transform(xTrain_num[0].reshape(1, -1)).shape

(787, 1004)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

(1, 1004)

In [317]:
print(np.array(xTrain_ohe).shape)

#xTrain_ohe = np.array(xTrain_ohe)
#xTrain_ohe

(787, 1, 3346)


In [296]:
print(xTrain_ohe.shape)
lens=set()
for x in xTrain_ohe:
    lens.add(len(x))
lens    

(787,)


{4016, 5020, 6024}

In [363]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils

def create_model_gridsearch_ohe(dropout=0.5):
    # create the model
    model = keras.Sequential()
    model.add(keras.layers.Embedding(6, 3, input_length=1004 ) )
    model.add(keras.layers.Conv1D(16, 3, activation='relu', padding='same'))
    model.add(keras.layers.Conv1D(16, 6, activation='relu', padding='same'))
    #model.add(keras.layers.Conv1D(8, 3, activation='relu', padding='same', input_shape=(3365, 1)))
    model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.Conv1D(64, 16, activation='relu'))
    model.add(keras.layers.Conv1D(64, 32, activation='relu'))
    #model.add(keras.layers.MaxPooling1D(2))
    #model.add(keras.layers.Conv1D(128, 16, activation='relu'))
    #model.add(keras.layers.Conv1D(32, 32, activation='relu'))
    model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(50, activation='relu'))
    #model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    optAdam = keras.optimizers.Adam()
    model.compile(optimizer=optAdam, loss='binary_crossentropy', metrics=['accuracy'])
    
    print(model.summary())
    
    return model


In [348]:
import pydot
import graphviz

ModuleNotFoundError: No module named 'graphviz'

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_55 (Embedding)     (None, 1004, 3)           18        
_________________________________________________________________
conv1d_164 (Conv1D)          (None, 1004, 16)          160       
_________________________________________________________________
conv1d_165 (Conv1D)          (None, 1004, 16)          1552      
_________________________________________________________________
max_pooling1d_127 (MaxPoolin (None, 502, 16)           0         
_________________________________________________________________
conv1d_166 (Conv1D)          (None, 487, 64)           16448     
_________________________________________________________________
conv1d_167 (Conv1D)          (None, 456, 64)           131136    
_________________________________________________________________
global_average_pooling1d_13  (None, 64)                0         
__________

In [335]:
yPred = model.predict(xTest_num)
#yPred_class = np.array([np.argmax(row) for row in yPred])

from sklearn.metrics import classification_report

print(classification_report( encoder.transform(yTest.Class.to_list()), yPred))


197/197 [==============================] - 1s 3ms/sample
              precision    recall  f1-score   support

           0       0.87      0.90      0.89        99
           1       0.89      0.87      0.88        98

    accuracy                           0.88       197
   macro avg       0.88      0.88      0.88       197
weighted avg       0.88      0.88      0.88       197



In [123]:
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_model_embedding, verbose=1)
# define the grid search parameters
batch_size = [32]
epochs = [5]
emb_in=[3,6,9] #6
emb_out=[2,3,4] #3
conv1_ks=[8,16,32] #16
conv1_w=[3,5]
conv2_ks=[8,16,32]
conv2_w=[7,9]
dense=[5,10,20,40]

param_grid = dict(batch_size=batch_size,
                 epochs=epochs,
                 emb_in=emb_in,
                 emb_out=emb_out,
                 conv1_ks=conv1_ks,
                 conv1_w=conv1_w,
                 conv2_ks=conv2_ks,
                 conv2_w=conv2_w,
                 dense=dense)



grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=2, n_jobs=-1, verbose=1)

grid_result = grid.fit(xTrain_oh, yTrain_enc)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 2 folds for each of 1296 candidates, totalling 2592 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/home/wehrmann/anaconda3/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 25.8min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 51.8min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 82.2min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 129.1min
[Parallel(n_jobs=-1)]: Done 2592 out of 2592 | elapsed: 141.0min finished
/home/wehrmann/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The defau

Epoch 1/5
787/787 [==============================] - 2s 3ms/sample - loss: 0.6987 - acc: 0.4828
Epoch 2/5
787/787 [==============================] - 2s 2ms/sample - loss: 0.6830 - acc: 0.7001
Epoch 3/5
787/787 [==============================] - 2s 2ms/sample - loss: 0.5211 - acc: 0.8463
Epoch 4/5
787/787 [==============================] - 2s 2ms/sample - loss: 0.2746 - acc: 0.9098
Epoch 5/5
787/787 [==============================] - 2s 2ms/sample - loss: 0.2315 - acc: 0.9250
Best: 0.921220 using {'batch_size': 32, 'conv1_ks': 16, 'conv1_w': 5, 'conv2_ks': 32, 'conv2_w': 9, 'dense': 40, 'emb_in': 6, 'emb_out': 3, 'epochs': 5}
0.739517 (0.136290) with: {'batch_size': 32, 'conv1_ks': 8, 'conv1_w': 3, 'conv2_ks': 8, 'conv2_w': 7, 'dense': 5, 'emb_in': 3, 'emb_out': 2, 'epochs': 5}
0.677255 (0.211338) with: {'batch_size': 32, 'conv1_ks': 8, 'conv1_w': 3, 'conv2_ks': 8, 'conv2_w': 7, 'dense': 5, 'emb_in': 3, 'emb_out': 3, 'epochs': 5}
0.673443 (0.207531) with: {'batch_size': 32, 'conv1_ks': 

In [124]:
grid_result.best_params_

{'batch_size': 32,
 'conv1_ks': 16,
 'conv1_w': 5,
 'conv2_ks': 32,
 'conv2_w': 9,
 'dense': 40,
 'emb_in': 6,
 'emb_out': 3,
 'epochs': 5}

In [125]:
model = grid_result.best_estimator_

In [126]:
yPred = model.predict(ohenc.transform(xTest_pad))
yPred_class = np.array([np.argmax(row) for row in yPred])

from sklearn.metrics import classification_report

print(classification_report( encoder.transform(yTest.Class.to_list()), yPred_class))

197/197 [==============================] - 0s 2ms/sample
              precision    recall  f1-score   support

           0       0.50      1.00      0.67        99
           1       0.00      0.00      0.00        98

    accuracy                           0.50       197
   macro avg       0.25      0.50      0.33       197
weighted avg       0.25      0.50      0.34       197



/home/wehrmann/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
